In [ ]:
import tempfile, urllib, zipfile, warnings, time, sys, os
import tensorflow as tf
import tensorflow_data_validation as tfdv
from __future__ import print_function
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format
tf.logging.set_verbosity(tf.logging.ERROR)
warnings.filterwarnings('ignore')
assert sys.version_info.major is 2, 'Oops, not running Python 2'

BASE_DIR = '/tmp/'
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
TRAIN_DATAX = os.path.join(DATA_DIR, '', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

ZIP, HEADERS = urllib.urlretrieve('https://storage.googleapis.com/tfx-colab-datasets/chicago_data.zip')
zipfile.ZipFile(ZIP).extractall(BASE_DIR)
zipfile.ZipFile(ZIP).close()

train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)
tfdv.visualize_statistics(train_stats)

In [ ]:
SCHEMA = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=SCHEMA)

In [ ]:
S = str(int(time.time()))
FILE_VERSION = '/tmp/payment_schema_' + S + '.pbtxt'
FILE_VERSION

In [ ]:
tfdv.write_schema_text(SCHEMA, FILE_VERSION)
READ_SCHEMA = tfdv.load_schema_text(FILE_VERSION)
ANOMALIES = tfdv.validate_statistics(statistics=train_stats, schema=READ_SCHEMA)
tfdv.display_anomalies(ANOMALIES)